In [1]:
!pip install transformers
!pip install jsonlines

     |████████████████████████████████| 573kB 3.5MB/s 
     |████████████████████████████████| 1.0MB 16.8MB/s 
     |████████████████████████████████| 3.7MB 16.5MB/s 
     |████████████████████████████████| 890kB 18.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=3003f3c9d60ae34e0fd28e96759070578ded5ad30352f0b11e211e7b2e38792e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import numpy as np
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
from google.colab import drive
import warnings

warnings.filterwarnings('ignore')
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
%cd drive/My \Drive/
!ls

/content/drive/My Drive
 accurate_test_output_test_Rake.jsonl   similarity2_labels.txt
 FEVER_confusion.png		        similarity3.jsonl
 final_test_labels.txt		        similarity3_labels.txt
 final_test_output.jsonl	        similarity.jsonl
'Getting started.pdf'		        similarity_labels.txt
 new_trained_model_10k.sav	        trained_model_10k.sav
 new_trained_model_15k.sav	        trained_model_15k.sav
 Rake_labels.txt		        trained_model20.sav
 sequence_matcher_labels.txt	        trained_model_5k.sav
 sequence_matcher_out.jsonl	        trained_model.sav
 shared_task_dev.jsonl		        train.jsonl
 similarity1.jsonl		        train_output_accurate.jsonl
 similarity1_labels.txt		        train_output.jsonl
 similarity2.jsonl


In [6]:
!head similarity.jsonl
!wc -l similarity.jsonl
!head train_output.jsonl

{"claim": "Colin Kaepernick became a starting quarterback during the 49ers 63rd season in the National Football League.", "sources": [["National Football League", 7], ["National Football League Draft", 19]], "evidence": ["league\tSports league\n9\tThe Super Bowl is among the biggest club sporting events in the world and individual Super Bowl games account for many of the most watched television programs in American history , all occupying the Nielsen 's Top 5 tally of the all-time most watched U.S. television broadcasts by 2015 .", "27"]}
{"claim": "Tilda Swinton is a vegan.", "sources": [["Tilda Swinton", 8], ["Snowpiercer", 11]], "evidence": ["Julia\tJulia (2008 film)\tI Am Love\tI Am Love (film)\tWe Need to Talk About Kevin\tWe Need to Talk About Kevin (film)\n12\t\n13\t\n14\tSwinton later starred in the dark romantic fantasy drama , Only Lovers Left Alive -LRB- 2014 -RRB- .", "15"]}
{"claim": "Fox 2000 Pictures released the film Soul Food.", "sources": [["Soul", 1], ["Soul II Soul"

In [0]:
# Load pre-trained model and tokenizer 
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [0]:
#define example and feature classes
class InputExample(object):
  def __init__(self, claim, ev, label=None):
    self.claim = claim
    self.ev = ev
    self.label = label

class InputFeatures(object):
    def __init__(self, claim_ids, ev_ids, label_id):
        self.claim_ids = claim_ids
        self.ev_ids = ev_ids
        self.label_id = label_id

In [0]:
#convert training data to InputExamples
import jsonlines
import re
def data_to_examples(data_file):
  examples = []
  data = jsonlines.open(data_file)
  #labels = jsonlines.open('train_output.jsonl')
  #for line,line2 in zip(data,labels):
  for index in range(845):
    line = data.read()
#   label = line2['label']
    claim = line['claim']
    ev=[]
    if line['evidences'] != 'null':
      for item in line['evidences']:
        #remove nonsense from sents
        clean = re.sub('\n+',' ',item)
        clean = re.sub('\s+',' ',clean)
        clean = re.sub('[a-zA-Z0-9]*\t[a-zA-Z0-9]*','',clean)
        clean = re.sub('-...-','',clean)
        if len(clean.split()) <= 512:
          ev.append(clean)
    else: #if evidence = null
      ev.append('null')
    examples.append(InputExample(claim,ev))#,label))
  return examples

In [0]:
test_ex = data_to_examples('accurate_test_output_test_Rake.jsonl')

In [0]:
#test_ex[28].ev[0] = test_ex[28].ev[0][:500]
print(test_ex[28].claim)
print(test_ex[28].ev)
print(len(test_ex[28].ev))

There is a show that is satire focused called The Daily Show.
['9 The Daily Show is the longest-running program on Comedy Central  counting all three tenures  , and has won 23 Primetime Emmy Awards . Comedy Central Comedy Central ', "6 Jon Stewart then took over as the host from January 11 , 1999 until August 6 , 2015 , making the show more strongly focused on politics and the national media , in contrast with the pop culture focus during Kilborn 's tenure . Jon Stewart Jon Stewart ", '8 Under different hosts , the show has been formally known as The Daily Show with Jon Stewart from 1999 until 2015 , and The Daily Show with Trevor Noah since 2015 . Jon Stewart Jon Stewart Trevor Noah Trevor Noah ']
3


In [0]:
def example_to_features(example):
  #tokenize and add [CLS]/[SEP] tokens
  claim_tokens = tokenizer.encode(example.claim, add_special_tokens=True)
  ev_tokens = []
  if example.ev ==[]:
    ev_tokens.append([101])
  for item in example.ev:
    ev_tokens.append(tokenizer.encode(item,add_special_tokens=True))
  
  return InputFeatures(claim_tokens,ev_tokens,example.label)

In [0]:
feat_list = []
for example in test_ex:
  features = example_to_features(example)
  feat_list.append(features)

In [0]:
print(feat_list[1].claim_ids)
print(feat_list[1].ev_ids)
print(feat_list[1].label_id)
print(len(feat_list[1].ev_ids))

[101, 1996, 2388, 1997, 9808, 3286, 2226, 8915, 22968, 2018, 2000, 22505, 5530, 1999, 2010, 14960, 1012, 102]
[[101, 1022, 2076, 2152, 2082, 1999, 3646, 8915, 22968, 2001, 7462, 2046, 1996, 2510, 2326, 2000, 2147, 1999, 1037, 4713, 1012, 102], [101, 1019, 2141, 1999, 9121, 7856, 2912, 8915, 22968, 2165, 2019, 3037, 1999, 2396, 1998, 7284, 2012, 2019, 2220, 2287, 2043, 2010, 2269, 3662, 2032, 2010, 2034, 6373, 3152, 1012, 102], [101, 1023, 8915, 22968, 2052, 4088, 5702, 4200, 2012, 1996, 13000, 2118, 1999, 3386, 1010, 2076, 2029, 2002, 2052, 4088, 2000, 10172, 2010, 2034, 2658, 2573, 1012, 13000, 2118, 13000, 2118, 102]]
None
3


In [0]:
#####BOTTLENECK STEP
#create BERT representations for each example
max_len = 432
BERT_examples = []
round = 0
for item in feat_list:
  #padding
  padded_claim = np.array(item.claim_ids + [0]*(max_len-len(item.claim_ids)))
  padded_claim = np.reshape(padded_claim,(1,max_len))
  padded_ev = np.array([sent + [0]*(max_len-len(sent)) for sent in item.ev_ids])
  
  #mask padded tokens
  claim_mask = np.where(padded_claim != 0,1,0)
  ev_mask = np.where(padded_ev != 0,1,0)

  #create matrix: each row represents one sentence
  padded_total = np.concatenate((padded_claim,padded_ev),axis=0)
  mask_total = np.concatenate((claim_mask,ev_mask))

  #convert to tensors
  input_ids = torch.tensor(padded_total)
  attention_mask = torch.tensor(mask_total)

  #get BERT embeddings
  with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

  #extract [CLS] embeddings and labels for input to classifier
  features = last_hidden_states[0][:,0,:].numpy()
  concatenated = features.flatten()
  BERT_examples.append(list(concatenated))
  round += 1
  print(round)

346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595


In [0]:
print(len(BERT_examples))

845


In [0]:
print(padded_claim)
print(padded_ev)

[[  101 13560  2956  2019  3059  1999  1037  2535  1012   102     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0 

In [0]:
#pad each example for uniform input size
max_len = 109824

padded_BERT = []
for example in BERT_examples:
  padded_BERT.append(example + [0]*(max_len-len(example)))

print(max_len)
print(len(padded_BERT[40]))

109824
109824


In [0]:
test_inputs = np.array([np.array(ex) for ex in padded_BERT])
#test_labels = [item.label_id for item in feat_list[:len(BERT_examples)]]
print(test_inputs.shape)

(845, 109824)


In [0]:
import pickle

# load the trained model from drive
loaded_model = pickle.load(open('new_trained_model_15k.sav', 'rb'))

In [0]:
#test classifier accuracy
predictions = loaded_model.predict(test_inputs)
print(predictions)

['SUPPORTS' 'REFUTES' 'SUPPORTS' 'SUPPORTS' 'SUPPORTS' 'SUPPORTS'
 'SUPPORTS' 'SUPPORTS' 'REFUTES' 'SUPPORTS' 'SUPPORTS' 'SUPPORTS'
 'REFUTES' 'REFUTES' 'SUPPORTS' 'REFUTES' 'SUPPORTS' 'SUPPORTS' 'SUPPORTS'
 'REFUTES' 'REFUTES' 'SUPPORTS' 'SUPPORTS' 'REFUTES' 'SUPPORTS' 'REFUTES'
 'SUPPORTS' 'SUPPORTS' 'SUPPORTS' 'SUPPORTS' 'SUPPORTS' 'SUPPORTS'
 'REFUTES' 'REFUTES' 'SUPPORTS' 'SUPPORTS' 'REFUTES' 'SUPPORTS' 'REFUTES'
 'SUPPORTS' 'SUPPORTS' 'SUPPORTS' 'SUPPORTS' 'REFUTES' 'REFUTES'
 'SUPPORTS' 'SUPPORTS' 'REFUTES' 'REFUTES' 'SUPPORTS' 'REFUTES' 'REFUTES'
 'SUPPORTS' 'SUPPORTS' 'SUPPORTS' 'SUPPORTS' 'SUPPORTS' 'SUPPORTS'
 'SUPPORTS' 'SUPPORTS' 'SUPPORTS' 'SUPPORTS' 'SUPPORTS' 'REFUTES'
 'SUPPORTS' 'REFUTES' 'REFUTES' 'REFUTES' 'REFUTES' 'REFUTES' 'REFUTES'
 'REFUTES' 'SUPPORTS' 'SUPPORTS' 'SUPPORTS' 'SUPPORTS' 'SUPPORTS'
 'SUPPORTS' 'SUPPORTS' 'SUPPORTS' 'REFUTES' 'SUPPORTS' 'SUPPORTS'
 'SUPPORTS' 'REFUTES' 'SUPPORTS' 'SUPPORTS' 'REFUTES' 'SUPPORTS'
 'SUPPORTS' 'SUPPORTS' 'REFUTES' 'SUP

In [0]:
labels = [line['label'] for line in jsonlines.open('train_output.jsonl')]
test_labels = labels[9350:10195]
print(len(test_labels))

845


In [0]:
filtered_p = []
filtered_t = []
for p_label, t_label in zip(predictions,test_labels):
  if t_label != 'NOT ENOUGH INFO':
    filtered_p.append(p_label)
    filtered_t.append(t_label)

print(len(filtered_p))
print(len(filtered_t))

import sklearn
sklearn.metrics.accuracy_score(filtered_t,filtered_p)

544
544


0.6636029411764706

In [0]:
nei = 0
for label in test_labels:
  if label == 'NOT ENOUGH INFO':
    nei +=1

print(nei)
print(nei/845)

301
0.3562130177514793


In [0]:
#test classifier accuracy
loaded_model.score(test_inputs, test_labels)

#'new_trained_model_10k.sav' got 0.7664 on 1507 test examples (gold evidence)
#'new_trained_model_10k.sav' got 0.4347 on 1500 test examples (our evidence)

#'new_trained_model_10k.sav' got 0.44 on similarity.jsonl
#'new_trained_model_10k.sav' got 0.43 on similarity1
#0.43 on similarity2
#0.44 on similarity3

#0.427 on Rake (.664 on verifiable claims)

0.4272189349112426

In [0]:
with open('Rake_labels.txt','a+') as out:
  for item in predictions:
    out.write(item)
    out.write('\n')

In [0]:
!wc -l 'Rake_labels.txt'

845 Rake_labels.txt


In [0]:
!head 'similarity3_labels.txt'

SUPPORTS
REFUTES
SUPPORTS
REFUTES
REFUTES
NOT ENOUGH INFO
SUPPORTS
SUPPORTS
REFUTES
REFUTES


In [0]:
data = jsonlines.open('final_test_output.jsonl')
labels = open('final_test_labels.txt')
output = open('final_test.jsonl')
index = 0
for line1 in :
  claim = line['claim']
  ev = line['evidence']
  label = 